<h2>Heartbeat Analysis</h2>
Syed Mahdi

In [2]:

import pandas as pd
import tensorflow as tf;
import keras;

In [4]:
path = "./heartbeat_dataset"  # Path to our dataset

In [29]:
set_a=pd.read_csv(path + "/set_a.csv")
print(f"Unique labels: {set_a['label'].unique()}")
set_a.head()

Unique labels: ['artifact' 'extrahls' 'murmur' 'normal' nan]


,dataset,fname,label,sublabel
0,a,set_a/artifact__201012172012.wav,artifact,NaN
1,a,set_a/artifact__201105040918.wav,artifact,NaN
2,a,set_a/artifact__201105041959.wav,artifact,NaN
3,a,set_a/artifact__201105051017.wav,artifact,NaN
4,a,set_a/artifact__201105060108.wav,artifact,NaN


In [19]:
set_a_timing=pd.read_csv(path + "/set_a_timing.csv")
set_a_timing.head()

,fname,cycle,sound,location
0,set_a/normal__201102081321.wav,1,S1,10021
1,set_a/normal__201102081321.wav,1,S2,20759
2,set_a/normal__201102081321.wav,2,S1,35075
3,set_a/normal__201102081321.wav,2,S2,47244
4,set_a/normal__201102081321.wav,3,S1,62992


In [27]:
set_b=pd.read_csv(path + "/set_b.csv")
print(f"Unique labels: {set_b['label'].unique()}")
set_b.head()

Unique labels: ['extrastole' 'murmur' 'normal' nan]


,dataset,fname,label,sublabel
0,b,set_b/Btraining_extrastole_127_1306764300147_C...,extrastole,NaN
1,b,set_b/Btraining_extrastole_128_1306344005749_A...,extrastole,NaN
2,b,set_b/Btraining_extrastole_130_1306347376079_D...,extrastole,NaN
3,b,set_b/Btraining_extrastole_134_1306428161797_C...,extrastole,NaN
4,b,set_b/Btraining_extrastole_138_1306762146980_B...,extrastole,NaN


In [ ]:
combined_dataset = set_a.append()